In [1]:
import ee
ee.Initialize()

In [2]:
import folium

EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print(f'Folium version: {folium.__version__}')

Folium version: 0.10.0


In [17]:
import pdb

def Mapdisplay(center, dicc, Tiles="OpenStreetMap", zoom_start=10):
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz
        

In [4]:
image1  = ee.Image(1)
print(image1, '\n')
print(image1.getInfo())

ee.Image({
  "type": "Invocation",
  "arguments": {
    "value": 1
  },
  "functionName": "Image.constant"
}) 

{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}


In [5]:
image2 = ee.Image(2)
image3 = ee.Image.cat([image1, image2])
print(image3, '\n')
print(image3.getInfo())

ee.Image({
  "type": "Invocation",
  "arguments": {
    "dstImg": {
      "type": "Invocation",
      "arguments": {
        "value": 1
      },
      "functionName": "Image.constant"
    },
    "srcImg": {
      "type": "Invocation",
      "arguments": {
        "value": 2
      },
      "functionName": "Image.constant"
    }
  },
  "functionName": "Image.addBands"
}) 

{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}, {'id': 'constant_1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}


In [6]:
# Create a multi-band image from a list of constants.
multiband = ee.Image([1, 2, 3])
print(multiband,'\n')
multiband.getInfo()

ee.Image({
  "type": "Invocation",
  "arguments": {
    "dstImg": {
      "type": "Invocation",
      "arguments": {
        "dstImg": {
          "type": "Invocation",
          "arguments": {
            "value": 1
          },
          "functionName": "Image.constant"
        },
        "srcImg": {
          "type": "Invocation",
          "arguments": {
            "value": 2
          },
          "functionName": "Image.constant"
        }
      },
      "functionName": "Image.addBands"
    },
    "srcImg": {
      "type": "Invocation",
      "arguments": {
        "value": 3
      },
      "functionName": "Image.constant"
    }
  },
  "functionName": "Image.addBands"
}) 



{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]},
  {'id': 'constant_1',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2},
   'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]},
  {'id': 'constant_2',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}

In [7]:
renamed = multiband.select(['constant', 'constant_1', 'constant_2'],
                           ['band1', 'band2', 'band3'])
print(renamed)

ee.Image({
  "type": "Invocation",
  "arguments": {
    "input": {
      "type": "Invocation",
      "arguments": {
        "dstImg": {
          "type": "Invocation",
          "arguments": {
            "dstImg": {
              "type": "Invocation",
              "arguments": {
                "value": 1
              },
              "functionName": "Image.constant"
            },
            "srcImg": {
              "type": "Invocation",
              "arguments": {
                "value": 2
              },
              "functionName": "Image.constant"
            }
          },
          "functionName": "Image.addBands"
        },
        "srcImg": {
          "type": "Invocation",
          "arguments": {
            "value": 3
          },
          "functionName": "Image.constant"
        }
      },
      "functionName": "Image.addBands"
    },
    "bandSelectors": [
      "constant",
      "constant_1",
      "constant_2"
    ],
    "newNames": [
      "band1",
      "ban

In [18]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
mapid = image.getMapId({
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.3
})

center = [38., -122.5]
Mapdisplay(center, {'Median Composite': mapid}, zoom_start=8)

In [20]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
ndwi = image.normalizedDifference(['B3', 'B5'])

ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiId = ndwiMasked.getMapId({
    'min': 0.5, 
    'max': 1,
    'palette': ['00FFFF', '0000FF']
})

center = [38, -122.5]
Mapdisplay(center, {'NDWI masked': ndwiId})

In [25]:
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)

mapRoi = image.clip(roi)
mapid = mapRoi.getMapId({
    'bands': ['B4'],
    'min': 0,
    'max': 0.3
})

center = [38, -122.5]
Mapdisplay(center, {'Mosaic': mapid})

In [26]:
# Load two 5-year Landsat 7 composites.
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Compute NDVI the hard way.
ndvi1999 = landsat1999.select('B4')\
                      .subtract(landsat1999.select('B3'))\
                      .divide(landsat1999.select('B4').add(landsat1999.select('B3')))

# Compute NDVI the easy way.
ndvi2008 = landsat2008.normalizedDifference(['B4', 'B3'])

#Vizualization
ndwiViz = {'min': 0, 'max': 1, 'palette': ['FF0000', '00FF00']}
Peru = ee.Geometry.Rectangle(-85, -20, -65,0) 

ndvi1999 = ndvi1999.clip(Peru)
ndvi2008 = ndvi2008.clip(Peru)

tiles = {
    "NDVI PERU 1999": ndvi1999.getMapId(ndwiViz), 
    "NDVI PERU 2008": ndvi2008.getMapId(ndwiViz)
}
center = [-12., -75.]
Mapdisplay(center, tiles)

In [28]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

bare = bare.updateMask(bare)
BareId = bare.getMapId()

tiles = {"Bare": BareId}
center = [37.7726, -122.3578]
Mapdisplay(center, tiles)

In [29]:
# Load a 2012 nightlights image.
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl2012.select('stable_lights')

# Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# Display the thresholded image as three distinct zones near Paris.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
center = [48.8683, 2.373]

zonesId = zones.getMapId({'min': 0, 'max': 3, 'palette': palette})
tiles = {"Zonas": zonesId}

Mapdisplay(center, tiles)

In [30]:

# Load a Landsat 8 top-of-atmosphere reflectance image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Convert the RGB bands to the HSV color space.
hsv = image.select(['B4', 'B3', 'B2']).rgbToHsv()

# Swap in the panchromatic band and convert back to RGB.
sharpened = ee.Image.cat([
  hsv.select('hue'), hsv.select('saturation'), image.select('B8')
]).hsvToRgb()

# Display the results.
tiles = {
    'rgb': image.getMapId({'bands': ['B4', 'B3', 'B2'], 
                           'min': 0, 
                           'max': 0.25, 
                           'gamma': [1.1, 1.1, 1]}),
    'pan-sharpened': sharpened.getMapId({'min': 0, 
                                         'max': 0.25, 
                                         'gamma': [1.3, 1.3, 1.3]})
}

center = [37.76664, -122.44829]
Mapdisplay(center, tiles, "Stamen Terrain")

In [31]:
# Load a Landsat 5 image and select the bands we want to unmix.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'];
image = ee.Image('LANDSAT/LT05/C01/T1/LT05_044034_20080214').select(bands)

# Define spectral endmembers.
urban = [88, 42, 48, 38, 86, 115, 59]
veg = [50, 21, 20, 35, 50, 110, 23]
water = [51, 20, 14, 9, 7, 116, 4]

# Unmix the image.
fractions = image.unmix([urban, veg, water])

# Display the results.
tiles = {
    'image': image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 128}),
    'unmixed': fractions.getMapId()
}

center = [37.5010, -122.1899]
Mapdisplay(center, tiles, "Stamen Terrain")

In [32]:
# Load a Landsat 8 image and display the thermal band.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Threshold the thermal band to find "hot" objects.
hotspots = image.select('B10').gt(303)

# Mask "cold" pixels.
hotspots = hotspots.updateMask(hotspots)

# Compute the number of pixels in each patch.
patchsize = hotspots.connectedPixelCount(256, False)

# Uniquely label the patches and visualize.
patchid = hotspots.connectedComponents(ee.Kernel.plus(1), 256)
    
# Display the results.
tiles = {
    "LST": image.getMapId({'bands': ['B10'], 'min': 270, 'max': 310}),
    "hotspots": hotspots.getMapId({'palette': 'FF0000'}),
    "patchsize": patchsize.getMapId(),
    "patches": patchid.getMapId()
}

center = [37.5010, -122.1899]
Mapdisplay(center, tiles, "Stamen Terrain")